In [1]:
import sys; sys.path.append('../')

import torch.nn as nn
from rl4co.utils.lightning import load_model_from_checkpoint

/home/hak/anaconda3/envs/torch200-py39/lib/python3.9/site-packages/torchrl/__init__.py:26: UserWarning: failed to set start method to spawn, and current start method for mp is fork.
  warn(
/home/hak/anaconda3/envs/torch200-py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
problem = 'cvrp'
size = '20'
model = f'am-{problem}{size}'

config = f'../../saved_checkpoints/{problem}{size}/{model}/config.yaml'
ckpt = f'../../saved_checkpoints/{problem}{size}/{model}/epoch_099.ckpt'

m_cvrp = load_model_from_checkpoint(config, ckpt)

Unused kwargs: {'params': {'total': 692608, 'trainable': 692608, 'non_trainable': 0}}


In [3]:
problem = 'tsp'
size = '20'
model = f'am-{problem}{size}'

config = f'../../saved_checkpoints/{problem}{size}/{model}/config.yaml'
ckpt = f'../../saved_checkpoints/{problem}{size}/{model}/epoch_099.ckpt'

m_tsp = load_model_from_checkpoint(config, ckpt)

Unused kwargs: {'params': {'total': 708608, 'trainable': 708608, 'non_trainable': 0}}


In [5]:
from copy import deepcopy

def transplant_model(source:nn.Module, 
                     target:nn.Module,
                     load_encoder:bool=True,
                     load_decoder:bool=True):
    source_policy = source.model.policy
    target_policy = target.model.policy
    
    # Transplant encoder's layer except init_embedding
    if load_encoder:
        source_encoder_params = source_policy.encoder.layers.state_dict()
        target_policy.encoder.layers.load_state_dict(source_encoder_params)
    
    if load_decoder:
        source_decoder = source_policy.decoder
        target_decoder = target_policy.decoder
        iter_ = zip(source_decoder.named_children(),
                    target_decoder.named_children())
        
        for (name, source), (_, target) in iter_:
            if name in ["env", "context", "dynamic_embedding"]:
                continue

            target.load_state_dict(source.state_dict())
            
    # if load_baseline:
        
transplant_model(m_tsp, m_cvrp)

In [39]:
mha1 = m_tsp.model.policy.encoder.layers[0][0].state_dict()['module.Wk']
mha2 = m_cvrp.model.policy.encoder.layers[0][0].state_dict()['module.Wk']

In [52]:
w1 = m_tsp.model.policy.decoder.logit_attention.project_out.state_dict()['weight']
w2 = m_cvrp.model.policy.decoder.logit_attention.project_out.state_dict()['weight']

In [53]:
w1 - w2

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])